In [ ]:

# Read the CSV file again (assuming the correct path this time)
df = pd.read_csv('updated_pressure_data_with_LatLon.csv')

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Splitting the DataFrame into separate ones for each layer
layers = df['Layer'].unique()  # Get all unique layers
layer_dfs = {layer: df[df['Layer'] == layer] for layer in layers}  # Dictionary to hold DataFrames for each layer

# Now, process each layer's DataFrame to calculate delta pressures
for layer, layer_df in layer_dfs.items():
    # Ensure the DataFrame is sorted by 'Time' for each group of 'Longitude' and 'Latitude'
    layer_df.sort_values(by='Time', inplace=True)

    # Group by 'Longitude' and 'Latitude' to operate on each unique location
    grouped = layer_df.groupby(['Longitude', 'Latitude'])

    # Initialize an empty DataFrame to store the results for this layer
    results_df = pd.DataFrame()

    # Loop through each group, calculate the delta pressure, and append to the results DataFrame
    for (longitude, latitude), group in grouped:
        # Find the reference pressure on the reference date (2017-01-01)
        reference_pressure = group[group['Time'] == pd.Timestamp('2017-01-01')]['Pressure'].iloc[0]

        # Calculate the delta pressure for each row in the group
        group['Delta_Pressure'] = group['Pressure'] - reference_pressure

        # Append the modified group to the results DataFrame
        results_df = pd.concat([results_df, group], ignore_index=True)

    # Save the results DataFrame for this layer to a CSV file
    output_path = f'delta_pressure_{layer.replace(" ", "_")}.csv'
    results_df.to_csv(output_path, index=False)

    # Print out the path to the saved file for verification
    print(f'Saved delta pressures for {layer} to {output_path}')

In [ ]:
# Read the CSV file again
df = pd.read_csv('/mnt/data/updated_pressure_data_with_LatLon.csv')

# Convert 'Time' to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Filter the DataFrame for Layer_13 only
layer_13_df = df[df['Layer'] == 'Layer_13']

# For calculating delta pressure, we need to compare each pressure value to the first available pressure for that specific location within Layer_13
# Group by Longitude and Latitude to handle each unique location
grouped = layer_13_df.groupby(['Longitude', 'Latitude'])

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame()

for name, group in grouped:
    # Ensure the group is sorted by time to get the earliest pressure value correctly
    group = group.sort(grouped.columns.drop('Pressure').tolist()).reset_index(drop=True)
    
    # The first pressure value in each group will be our reference
    reference_pressure = group.iloc[0]['Pressure']
    
    # Calculate delta pressure for each row in the group
    group['Delta_Pressure'] = group['Pressure'] - reference_pressure
    
    # Append the group with calculated delta pressures to the results DataFrame
    results_df = pd.concat([results_df, group], ignore_index=True)

# Now, results_df contains all data for Layer_13 with an additional column for delta pressure
results_df.head()

In [ ]:
import pandas as pd
import pyproj
import numpy as np


# Read the CSV file again
df = pd.read_csv('updated_pressure_data_with_LatLon.csv')

df['Time'] = pd.to_datetime(df['Time'])
layer_13_df = df[df['Layer'] == 'Layer_13']

# Group by Longitude and Latitude for unique locations within Layer_13
grouped = layer_13_df.groupby(['Longitude', 'Latitude'])

# Initialize an empty DataFrame for results
delta_pressure_df = pd.DataFrame()

# Iterate through each group
for (longitude, latitude), group in grouped:
    # Filter to get the reference pressure on 2017-01-01 for each location
    reference_pressure_row = group[group['Time'] == pd.Timestamp('2017-01-01')]
    if not reference_pressure_row.empty:
        reference_pressure = reference_pressure_row.iloc[0]['Pressure']
        # Calculate delta pressure for each row in the group by subtracting the reference pressure
        group['Delta_Pressure'] = group['Pressure'] - reference_pressure
        # Append the processed group to the results DataFrame
        delta_pressure_df = pd.concat([delta_pressure_df, group])

# Resetting index of the final DataFrame
delta_pressure_df.reset_index(drop=True, inplace=True)

# Save the results DataFrame for this layer to a CSV file
output_path = f'delta_pressure_layer13.csv'
delta_pressure_df.to_csv(output_path, index=False)


# Checking the first few rows of the final DataFrame
delta_pressure_df.head()

In [4]:
import pandas as pd
from pathlib import Path

# Function to calculate delta pressure
def calculate_delta_pressure(group):
    # Sort group by time
    group = group.sort_values('Time')
    # Get the reference pressure from the first date (assumed to be the earliest in the group)
    reference_pressure = group.iloc[0]['Pressure']
    # Calculate delta pressure
    group['Delta_Pressure'] = group['Pressure'] - reference_pressure
    return group

# Read CSV
df_path = 'updated_pressure_data_with_LatLon.csv'
df = pd.read_csv(df_path)

# Convert 'Time' to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Get unique layers
layers = df['Layer'].unique()


# Calculate delta pressure for each layer and save to a separate CSV
for layer in layers:
    layer_df = df[df['Layer'] == layer]
    # Group by 'Longitude' and 'Latitude' to handle each location separately
    grouped = layer_df.groupby(['Longitude', 'Latitude'])
    # Apply the calculate_delta_pressure function to each group
    delta_df = grouped.apply(calculate_delta_pressure).reset_index(drop=True)
    # Save to CSV
    output_file = f'delta_pressure_{layer}.csv'
    delta_df.to_csv(output_file, index=False)



[WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_1.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_10.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_11.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_12.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_13.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_14.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_15.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_16.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_2.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_3.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_4.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_5.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_6.csv'),
 WindowsPath('/mnt/data/delta_pressures/delta_pressure_Layer_7.csv'),
 WindowsPath(

In [5]:
import pandas as pd

# Function to calculate delta pressure
def calculate_delta_pressure(group):
    # Sort group by time
    group = group.sort_values('Time')
    # Get the reference pressure from the first date (assumed to be the earliest in the group)
    reference_pressure = group.iloc[0]['Pressure']
    # Calculate delta pressure
    group['Delta_Pressure'] = group['Pressure'] - reference_pressure
    return group

# Read CSV
df_path = 'updated_pressure_data_with_LatLon.csv'
df = pd.read_csv(df_path)

# Convert 'Time' to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Get unique layers
layers = df['Layer'].unique()

# Initialize a DataFrame to store all the delta pressures
all_layers_delta_df = pd.DataFrame()

# Calculate delta pressure for each layer and append to the all_layers_delta_df DataFrame
for layer in layers:
    layer_df = df[df['Layer'] == layer]
    # Group by 'Longitude' and 'Latitude' to handle each location separately
    grouped = layer_df.groupby(['Longitude', 'Latitude'])
    # Apply the calculate_delta_pressure function to each group
    delta_df = grouped.apply(calculate_delta_pressure).reset_index(drop=True)
    # Append to the all_layers_delta_df DataFrame
    all_layers_delta_df = pd.concat([all_layers_delta_df, delta_df], ignore_index=True)

# Save the combined DataFrame to a CSV file without headers for subsequent rows
output_file = 'delta_pressure_all_layers.csv'
all_layers_delta_df.to_csv(output_file, index=False)

all_layers_delta_df.head(), output_file

(        Time  Pressure    Layer   Longitude   Latitude  Delta_Pressure
 0 2017-01-01   7952.25  Layer_1 -103.777418  31.533703             0.0
 1 2017-01-01   7970.05  Layer_1 -103.760446  31.534198             0.0
 2 2017-01-01   7658.27  Layer_1 -103.998653  31.541595             0.0
 3 2017-01-01   7754.00  Layer_1 -103.981681  31.542119             0.0
 4 2017-01-01   7828.24  Layer_1 -103.964708  31.542642             0.0,
 'delta_pressure_all_layers.csv')